In [2]:
import csv
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import interactive,interact
import plotly.figure_factory as ff
import plotly.plotly as py

### read in modules
import csv
L = csv.reader(open('SC1_sig_modules_22122016.txt','rU'),delimiter='\t')
sig_modules={}
for r in L:
    teamID=int(r[0])
    if teamID not in sig_modules:
        sig_modules[teamID]={}
    if r[2] not in sig_modules[teamID]:
        sig_modules[teamID][r[2]]={}
    sig_modules[teamID][r[2]][int(r[3])]=r[4:]

### list of sig modules 
df =pd.read_csv('SC1_sig_module_gwas_cat_14032017.txt')

### read in networks and make a esgelist of all
import networkx as nx
import os
# G= nx.read_edgelist('networks/3_signal_omnipath_directed.txt',delimiter='\t',data=(('weight',float),))
G=nx.Graph()
for f in os.listdir('networks/'):
    if f.endswith('.txt'):
        g= nx.read_edgelist('networks/'+f,delimiter='\t',data=(('weight',float),))
        G.add_edges_from(g.edges(data=True))



In [3]:
### gene score & gene sig or not < 10-8
path_to_gene_scores= '/Users/schoobdar/Documents/DreamChallenge/code/baselines/PASCAL_geneScore_precomputed/gwas_collection_dmi_vitalit/gene_scores/16-08-10_leaderboardFinal_sum/scores/'
gwas_list =[]
L = csv.reader(open('gwas_final_v1.txt','rU'),delimiter=',') 
for i in L:
    a ='.'.join( i[0].split('.')[:-2])
    gwas_list.append(a)
L = csv.reader(open('gwas_leaderboard_v2.txt','rU'),delimiter=',')
for i in L:
    a ='.'.join( i[0].split('.')[:-2])
    gwas_list.append(a)
gene_scores ={}
for i in xrange(len(gwas_list)):
    p = gwas_list[i]
    gene_scores[p]={}
    gene_file = os.path.basename(p) + ".sum.genescores.txt"
    L = csv.reader(open(os.path.join(path_to_gene_scores,gene_file), 'rU'), delimiter='\t')
    L.next()
    for a in L:
        geneID =str.upper(a[4])
        gene_scores[p][geneID]=float(a[-2])


In [4]:
### with lightning

import os
from lightning import Lightning

from numpy import random, asarray, linspace, corrcoef
from colorsys import hsv_to_rgb
from sklearn import datasets
import networkx as nx

def plot_net_1(g,scores):
    lgn = Lightning(ipython=True, local='http://public.lightning-viz.org')
    mat = nx.adjacency_matrix(g).todense()
    labels = [str(u[0]) for u in g.nodes(data=True)]
    d = g.degree().values()
    return lgn.force(mat, group=scores,size=(asarray(d) + 1.5)*1,labels=labels)

In [5]:
import plotly.plotly as py
from plotly.graph_objs import *
import numpy as np
def plot_net(g,colors,gs_name):
    pos=nx.fruchterman_reingold_layout(g)
    N = nx.number_of_nodes(g)
    labels = [str(u[0]) for u in g.nodes(data=True)]
    d = g.degree().values()
    
    Xv=[pos[k][0] for k in g.nodes()]
    Yv=[pos[k][1] for k in g.nodes()]
    Xed=[]
    Yed=[]
    for edge in nx.edges(g):
        Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
        Yed+=[pos[edge[0]][1],pos[edge[1]][1], None] 

    ### plotting setting

    trace3=Scatter(x=Xed,
                   y=Yed,
                   mode='lines',
                   line=Line(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   )

    trace4=Scatter(x=Xv,
                   y=Yv,
                   mode='markers',
                   name='net',
                   marker=Marker(symbol='dot',
                                 size=20,#(asarray(d) + 2)*1, 
                                 color=colors,
                                  colorscale='Viridis',
                                  showscale=True,
                                  colorbar = dict(
                                             title = 'Gene association, -log(pvalue)',
                                             titleside = 'top',
                                             tickmode = 'array',
#                                              ticktext = color_map.keys(),
#                                              tickvals = list(set(colors)),
                                             ticks = 'outside'
                                         ),
                                 line=Line(color='rgb(50,50,50)', width=0.5)
                                 ),
                   text=labels,
                   hoverinfo='text'
                   )

    annot="Size of nodes is proportional to gene degree in the network."

    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='' 
              )

    width=800
    height=800
    layout=Layout(title= gs_name +" trait module",  
        font= Font(size=12),
        showlegend=False,
        autosize=False,
        width=width,
        height=height,
        xaxis=XAxis(axis),
        yaxis=YAxis(axis),          
        margin=Margin(
            l=40,
            r=40,
            b=85,
            t=100,
        ),
        hovermode='closest',
        annotations=Annotations([
               Annotation(
               showarrow=False, 
                text='This igraph.Graph has the Kamada-Kawai layout',  
                xref='paper',     
                yref='paper',     
                x=0,  
                y=-0.1,  
                xanchor='left',   
                yanchor='bottom',  
                font=Font(
                size=14 
                )     
                )
            ]),           
        )

    data1=Data([trace3, trace4])
    fig1=Figure(data=data1, layout=layout)
    fig1['layout']['annotations'][0]['text']=annot
    return py.iplot(fig1, filename='trait-module-nx')


In [6]:
import networkx as nx
import plotly.plotly as py
from plotly.graph_objs import *


def trait_sim_graph():
    g = nx.read_graphml('trait_similarity_network.graphml')
    pos=nx.fruchterman_reingold_layout(g)
    N = nx.number_of_nodes(g)
    labels = [str(u[1]['name']) for u in g.nodes(data=True)]

    Xv=[pos['n'+str(k)][0] for k in range(N)]
    Yv=[pos['n'+str(k)][1] for k in range(N)]
    Xed=[]
    Yed=[]
    for edge in nx.edges(g):
        Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
        Yed+=[pos[edge[0]][1],pos[edge[1]][1], None] 
    import csv
    L= csv.reader(open('gwasCollection_dream11_traitGroups.txt','rU'),delimiter='\t')
    
    L.next()
    gwas_categories = {}
    for i in L:
        gwas_categories[i[5]]= i[1]
    gcat = {}
    for u in g.nodes():
        gcat[u]=gwas_categories[g.node[u]['name']] if g.node[u]['name'] in gwas_categories else None
    nx.set_node_attributes(g,'cat',gcat)

    rgb_colors = ['rgb(107,142,35)','rgb(0,206,209)','rgb(93, 164, 214)', 'rgb(255, 144, 14)',  'rgb(44, 160, 101)', 'rgb(255, 65, 54)','rgb(128,0,0)','rgb(255,215,0)','rgb(0,0,0)']
    color_map = { list(set(gwas_categories.values()))[i]:rgb_colors[i] for i in range(len(rgb_colors)-1) }
    color_map[None]= 'rbg(0,0,0)' 
    colors = [color_map[u[1]['cat']] for u in g.nodes(data=True)]

    ### plotting setting

    trace3=Scatter(x=Xed,
                   y=Yed,
                   mode='lines',
                   line=Line(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   )

    trace4=Scatter(x=Xv,
                   y=Yv,
                   mode='markers',
                   name='net',
                   marker=Marker(symbol='dot',
                                 size=20, 
                                 color=colors,
                                 colorscale=[[i/float(len(rgb_colors)-1),rgb_colors[i]] for i in range(len(rgb_colors))],
#                                  colorscale=[[0, 'rgb(166,206,227)'], [0.25, 'rgb(31,120,180)'], [0.45, 'rgb(178,223,138)'], [0.65, 'rgb(51,160,44)'], [0.85, 'rgb(251,154,153)'], [1, 'rgb(227,26,28)']],
                                 showscale=True,
                                 colorbar = dict(
                                            title = 'Trait type',
                                            titleside = 'top',
                                            tickmode = 'array',
                                            ticktext = color_map.keys(),
                                            tickvals = rgb_colors,
                                            ticks = 'outside'
                                        ),
                                 line=Line(color='rgb(50,50,50)', width=0.5)
                                 ),
                   text=labels,
                   hoverinfo='text'
                   )

    annot="This networkx.Graph has the Fruchterman-Reingold layout<br>Code:"+\
    "<a href='http://nbviewer.ipython.org/gist/empet/07ea33b2e4e0b84193bd'> [2]</a>"

    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='' 
              )

    width=800
    height=800
    layout=Layout(title= " Trait network showing similarity between GWAS traits based on overlap of associated modules.",  
        font= Font(size=12),
        showlegend=False,
        autosize=False,
        width=width,
        height=height,
        xaxis=XAxis(axis),
        yaxis=YAxis(axis),          
        margin=Margin(
            l=40,
            r=40,
            b=85,
            t=100,
        ),
        hovermode='closest',
        annotations=Annotations([
               Annotation(
               showarrow=False, 
                text='This igraph.Graph has the Kamada-Kawai layout',  
                xref='paper',     
                yref='paper',     
                x=0,  
                y=-0.1,  
                xanchor='left',   
                yanchor='bottom',  
                font=Font(
                size=14 
                )     
                )
            ]),
        plot_bgcolor='rgb(243, 243, 243)',
        paper_bgcolor='rgb(243, 243, 243)',
        
        )

    data1=Data([trace3, trace4])
    fig1=Figure(data=data1, layout=layout)
    fig1['layout']['annotations'][0]['text']=annot
    return py.iplot(fig1, filename='trait-network-nx')

trait_sim_graph()


In [7]:

caption = widgets.Label(value='List of trait module by trait type and network:',size=50)
display(caption)
items = ['All']+sorted(df['type'].unique().tolist())



a_slider = widgets.IntSlider(min=3, max=100, step=1, value=3, description='Module Size')

b_select= widgets.Dropdown(
    options=items,
    value='All',
    description='Trait Type:',
    disabled=False,
)

items_traitGroup = ['All']+sorted(df['traitGroup'].unique().tolist())


c_select= widgets.Dropdown(
    options= items_traitGroup,
    value='All',
    description='Trait Group:',
    disabled=False,
)

d_text = widgets.IntText(
    value=None,
    description='Module ID:',
    disabled=False
)

def handle_range_change(change):
    c_select.options=['All']+sorted(df[df['type']==b_select.value]['traitGroup'].unique().tolist())
    d_text.value=0
b_select.observe(handle_range_change, names='value')

def handle_module_id_change(change):
    d = d_text.value
    if d>0:
        c_select.value=df.iloc[d,:]['traitGroup']
        b_select.value = df.iloc[d,:]['type']
d_text.observe(handle_module_id_change,names='value')



def view3(x='',z='',y=3,d=''):
    if d>0:
        ind=df.iloc[d,[1,4,8]].tolist()
        selected_m = sig_modules[ind[0]][ind[1]][ind[2]]
        sg = G.subgraph(selected_m)
        gwas_name = df.iloc[d,:]['trait']
        gene_association = [-1*np.log(gene_scores[df.iloc[d,:].gwas_name][u]) if u in gene_scores[df.iloc[d,:].gwas_name] else 0 for u in sg.nodes()]
        return  plot_net(sg,gene_association,gwas_name)
#         return plot_net_1(sg,gene_association)
    if x=='All': 
        res= df[df['module_size']>y].groupby(['traitGroup'])['pval'].count().reset_index()
        res.rename(columns={'pval':'Total Number','traitGroup':'Trait Type'}, inplace= True)
        return res
    res = df[(df['type']==x)&(df['traitGroup']==z)& (df['module_size']>y)].iloc[:,[4,6,9,7]].reset_index().sort_values(['pval','module_size'])
    res =res.rename(columns={'module_size':'Module Size','pval':'Pvalue','index':'Module ID','net':'Network'})
#     table = ff.create_table(res)
    return res #py.iplot(table, filename='pandas_table')

interact(view3,y=a_slider,x=b_select,z=c_select,d=d_text)



A Jupyter Widget

A Jupyter Widget

<function __main__.view3>